## Part I: Introduction to TensorFlow


In [2]:
import tensorflow as tf
import numpy as np


In [3]:
from sklearn.datasets import load_boston
boston_X, boston_y = load_boston(return_X_y=True)
nobs, nfeatures = boston_X.shape
boston_y = boston_y.reshape(nobs, 1)

## Penalized Least Squares Regression
### Let's use tensor flow to implement a gradient descent solution for the penalized least squares regression problem


In [4]:
# first setup data and optimization parameters and variables

X = tf.constant(np.c_[np.ones(nobs), boston_X], dtype=tf.float32, name="X")
y = tf.constant(boston_y, dtype=tf.float32, name="y")

lam = 1.0
penalty_vec = np.vstack([[0.0], np.full((nfeatures, 1), lam)])

learning_rate = 0.000001
nepochs = 1000

stddev = 2 / np.sqrt(nfeatures)

theta = tf.Variable(tf.truncated_normal([nfeatures + 1, 1], stddev=stddev), name="theta")

f = tf.matmul(X, theta, name="f")
r = y - f
loss = 0.5 * tf.reduce_mean(tf.square(r), name="loss")
obj = loss + 0.5 * tf.matmul(tf.transpose(theta), penalty_vec * theta)

grad = - (tf.matmul(tf.transpose(X), r / nobs)) + penalty_vec * theta
gradnorm = tf.norm(grad)


update = tf.assign(theta, theta - learning_rate * grad)

In [5]:
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(nepochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "obj=", obj.eval(), "grad_norm=", gradnorm.eval())
        sess.run(update)
        
    solution = theta.eval()
    print(solution)
    print("Training set errror", loss.eval())

Epoch 0 obj= [[ 48131.44140625]] grad_norm= 169148.0
Epoch 100 obj= [[ 125.82846069]] grad_norm= 1046.76
Epoch 200 obj= [[ 97.02219391]] grad_norm= 277.868
Epoch 300 obj= [[ 90.41902161]] grad_norm= 245.32
Epoch 400 obj= [[ 84.77169037]] grad_norm= 230.105
Epoch 500 obj= [[ 79.79210663]] grad_norm= 216.248
Epoch 600 obj= [[ 75.38844299]] grad_norm= 203.498
Epoch 700 obj= [[ 71.4835968]] grad_norm= 191.759
Epoch 800 obj= [[ 68.01160431]] grad_norm= 180.948
Epoch 900 obj= [[ 64.91586304]] grad_norm= 170.986
[[ 0.67546397]
 [-0.67017102]
 [ 0.46850663]
 [-0.08006571]
 [ 0.57550502]
 [ 0.02331622]
 [ 0.75861716]
 [ 0.190152  ]
 [ 0.06409112]
 [-0.20756727]
 [ 0.02193321]
 [-0.09849752]
 [ 0.00208634]
 [-0.33416644]]
Training set errror 61.2541


### Now, instead of providing gradients directly, use numerical differentiation

In [6]:
auto_grad = tf.gradients(obj, [theta])[0]
gradnorm = tf.norm(auto_grad)
auto_update = tf.assign(theta, theta - learning_rate * auto_grad)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(nepochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "obj=", obj.eval(), "grad_norm=", gradnorm.eval())
        sess.run(auto_update)
    solution = theta.eval()
    print(solution)
    print("Training set error", loss.eval())

Epoch 0 obj= [[ 34935.296875]] grad_norm= 144182.0
Epoch 100 obj= [[ 115.98401642]] grad_norm= 868.475
Epoch 200 obj= [[ 96.95607758]] grad_norm= 210.812
Epoch 300 obj= [[ 93.12779236]] grad_norm= 189.115
Epoch 400 obj= [[ 89.67462158]] grad_norm= 182.618
Epoch 500 obj= [[ 86.44981384]] grad_norm= 176.536
Epoch 600 obj= [[ 83.43518066]] grad_norm= 170.714
Epoch 700 obj= [[ 80.61523438]] grad_norm= 165.135
Epoch 800 obj= [[ 77.97582245]] grad_norm= 159.786
Epoch 900 obj= [[ 75.5039978]] grad_norm= 154.652
[[-0.4660655 ]
 [-0.44155982]
 [-0.18564823]
 [ 0.74719751]
 [ 0.23611598]
 [-0.30666822]
 [ 0.49736616]
 [-0.27598262]
 [ 0.0386317 ]
 [-0.55132735]
 [ 0.04128872]
 [ 0.49269521]
 [ 0.05408758]
 [-0.51907086]]
Training set error 72.1461


### Finally, use an optimizer provided by tensor flow

In [7]:
opt = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
train_op = opt.minimize(obj)

with tf.Session() as sess:
    sess.run(init)
    sess.run(train_op)
    solution = theta.eval()
    print(solution)
    print("Training set error", loss.eval())

[[-0.28548339]
 [-0.22648376]
 [-0.46971554]
 [-0.42798772]
 [ 0.29443103]
 [ 0.1560858 ]
 [-0.83637637]
 [-1.11538863]
 [-1.05088902]
 [ 0.21619606]
 [ 0.94561756]
 [ 0.49943584]
 [-0.30007377]
 [-0.12833865]]
Training set error 28208.4


# Part I: Implement Gradient Descent Linear SVM Classifier with Tensor Flow

## Following the example above, re-implement your Gradient Descent Linear SVM classifier using Tensor Flow.


### First implement by providing its gradients


In [8]:
X = tf.constant(np.c_[np.ones(nobs), boston_X], dtype=tf.float32, name="X")
y = tf.constant(boston_y, dtype=tf.float32, name="y")

lam = 1.0
penalty_vec = np.vstack([[0.0], np.full((nfeatures, 1), lam)])

learning_rate = 0.000001
nepochs = 1000

stddev = 2 / np.sqrt(nfeatures)

theta = tf.Variable(tf.truncated_normal([nfeatures + 1, 1], stddev=stddev), name="theta")

f = tf.matmul(X, theta, name="f")

yf = y*f
u=(1.0-yf)

condition = tf.greater(u, 0.)
pos_part = tf.where(condition, u, tf.zeros_like(u))

loss=tf.reduce_mean(pos_part, name="loss")
obj = loss + 0.5 * tf.matmul(tf.transpose(theta), penalty_vec * theta)

condition2 = tf.less(yf, 1.)
t = tf.where(condition2, -tf.ones_like(yf), tf.zeros_like(yf))


ty=t*y

grad= (tf.matmul(tf.transpose(X), ty/nobs)) + penalty_vec * theta
gradnorm = tf.norm(grad)


update = tf.assign(theta, theta - learning_rate * grad)

In [9]:
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(nepochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "obj=", obj.eval(), "grad_norm=", gradnorm.eval())
        sess.run(update)
        
    solution = theta.eval()
    print(solution)
    print("Training set errror", loss.eval())

Epoch 0 obj= [[ 4459.87646484]] grad_norm= 11980.0
Epoch 100 obj= [[ 4.5340066]] grad_norm= 387.767
Epoch 200 obj= [[ 1.0660162]] grad_norm= 1.46015
Epoch 300 obj= [[ 1.06580532]] grad_norm= 1.46
Epoch 400 obj= [[ 1.06559432]] grad_norm= 1.45986
Epoch 500 obj= [[ 1.06538343]] grad_norm= 1.45971
Epoch 600 obj= [[ 1.06517231]] grad_norm= 1.45957
Epoch 700 obj= [[ 1.06496155]] grad_norm= 1.45943
Epoch 800 obj= [[ 1.06475055]] grad_norm= 1.45928
Epoch 900 obj= [[ 1.06453979]] grad_norm= 1.45914
[[ 0.43312255]
 [ 0.19515757]
 [-0.48010713]
 [-0.17471494]
 [-0.36800322]
 [-0.05611996]
 [-0.97037131]
 [ 0.37912083]
 [ 0.59237027]
 [-0.09108316]
 [ 0.02185793]
 [ 0.48559609]
 [ 0.06975992]
 [-0.07258014]]
Training set errror 0.0


### Next, change it to use numerical differentiation


In [10]:
auto_grad = tf.gradients(obj, [theta])[0]
gradnorm = tf.norm(auto_grad)
auto_update = tf.assign(theta, theta - learning_rate * auto_grad)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(nepochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "obj=", obj.eval(), "grad_norm=", gradnorm.eval())
        sess.run(auto_update)
    solution = theta.eval()
    print(solution)
    print("Training set error", loss.eval())

Epoch 0 obj= [[ 4394.24365234]] grad_norm= 11980.0
Epoch 100 obj= [[ 70.72247314]] grad_norm= 729.186
Epoch 200 obj= [[ 30.6693058]] grad_norm= 544.161
Epoch 300 obj= [[ 9.26455116]] grad_norm= 322.762
Epoch 400 obj= [[ 3.13280988]] grad_norm= 165.625
Epoch 500 obj= [[ 1.86560357]] grad_norm= 93.2578
Epoch 600 obj= [[ 1.44370627]] grad_norm= 46.3753
Epoch 700 obj= [[ 1.2312324]] grad_norm= 31.8941
Epoch 800 obj= [[ 1.131387]] grad_norm= 20.11
Epoch 900 obj= [[ 1.09094965]] grad_norm= 20.108
[[-0.04730881]
 [-0.34313971]
 [ 0.08574782]
 [ 0.01171829]
 [ 0.82060707]
 [ 0.09238216]
 [-0.66020775]
 [ 0.34411767]
 [-0.0950948 ]
 [ 0.46359235]
 [-0.05423754]
 [ 0.25593981]
 [ 0.33474696]
 [ 0.57677305]]
Training set error 0.00122579


### Lastly, use the included gradient descent optimizer

In [11]:
opt = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
train_op = opt.minimize(obj)

with tf.Session() as sess:
    sess.run(init)
    sess.run(train_op)
    solution = theta.eval()
    print(solution)
    print("Training set error", loss.eval())

[[ 0.44889131]
 [ 0.92554706]
 [-0.09448642]
 [-0.14388104]
 [ 0.81002623]
 [-0.07992097]
 [-0.16729937]
 [ 0.47532859]
 [-0.71648061]
 [ 0.52153224]
 [-0.13271137]
 [ 0.00708812]
 [-0.67695987]
 [ 0.42074496]]
Training set error 5964.36
